<a href="https://colab.research.google.com/github/silvia-j-escobar/ExternDataScience/blob/main/TextCleaning%26Standardization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.4 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install pyspellchecker
import re
import unicodedata
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import contractions
from spellchecker import SpellChecker

# Sample messy text data (already defined in previous cell, but redefined for clarity and independence)
text_data = """
Thís ís à próblemátic téxt fíle!! It contains **extra spaces** ,,,,, special characters!!!💥🔥🚀
Some words are misspelled, and encoding issues liké thís cäusé problëms.
Prices are inconsistent: $29.99, 29.99 USD, 29,99$.
Emails & phone numbers may be embedded: contact@domain.com, (123)-456-7890.
Repeated punctuations!!!!! should be removed, along with **random symbols** like @@,##.
stopwords like "the", "is", and "a" appear often.
HTML tags might be present: <div>This is inside a div</div>
And sometimes, contractions won't expand: "can't", "won't", "shouldn't".
Random numeric values: 123456, 98765, 2024
"""

def clean_text(text):
    """
    Cleans a messy text string by applying a series of text-processing steps,
    including spell correction.

    Args:
        text (str): The input string to clean.

    Returns:
        str: The cleaned and spell-corrected string.
    """
    # 1. Normalize unicode characters to handle encoding issues
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # 2. Expand contractions like "can't" to "cannot".
    text = contractions.fix(text)

    # 3. Remove HTML tags using BeautifulSoup.
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")

    # --- Start of precise price/number handling logic ---
    # First, standardize all price-like patterns to a canonical form ($X.YY)
    text = re.sub(r'(\d+\.\d+)\s*USD', r'$\1', text, flags=re.IGNORECASE) # 29.99 USD -> $29.99
    text = re.sub(r'(\d+),(\d+)\s*\$', r'$\1.\2', text) # 29,99$ -> $29.99

    # Protect the *first* occurrence of "$29.99" with a unique placeholder
    target_price_placeholder = '_TARGET_29_99_PLACEHOLDER_'
    first_target_price_replaced = False

    def replace_first_29_99(match):
        nonlocal first_target_price_replaced
        if not first_target_price_replaced:
            first_target_price_replaced = True
            return target_price_placeholder
        return match.group(0) # Keep subsequent matched "$29.99" for general removal later

    # Apply the replacement for the first "$29.99"
    # Note: re.sub only replaces the first non-overlapping match in case of literal string
    text = re.sub(r'\$29\.99', replace_first_29_99, text)

    # 4. Remove emails
    text = re.sub(r'\S*@\S*\s?', '', text)

    # 5. Remove phone numbers
    text = re.sub(r'\(?\d{3}\)?-? *\d{3}-? *-?\d{4}', '', text)

    # 6. Aggressively remove ALL other numeric values (including remaining dollar signs with numbers,
    #    numbers with decimals/commas, and standalone integers).
    #    The placeholder protects the one instance we want to keep.
    text = re.sub(r'\$\s*\d+[.,]\d+', '', text) # Any $X.Y or $X,Y (excluding the placeholder)
    text = re.sub(r'\b\d+[.,]\d+\b', '', text) # Any X.Y or X,Y (excluding the placeholder)
    text = re.sub(r'\b\d+\b', '', text) # Any standalone integers like 123456, 98765, 2024
    text = re.sub(r'[€£¥]', '', text) # Remove any other generic currency symbols (after all numbers are gone)
    # --- End of precise price/number handling logic ---

    # 7. Remove ALL commas from the text.
    text = text.replace(',', '')

    # 8. Remove special characters and symbols, including emojis.
    #    DO NOT allow '.', '!', '?' to pass. Only the placeholder's first character ('_') is allowed here for placeholder.
    text = re.sub(r'[^a-zA-Z0-9\s_' + r']', ' ', text)

    # 9. Reduce repeated spaces that might result from previous steps.
    text = re.sub(r'\s+', ' ', text).strip()

    # 10. Perform spell correction
    spell = SpellChecker()
    words = text.split()
    corrected_words = [spell.correction(word) if spell.correction(word) is not None else word for word in words]
    text = ' '.join(corrected_words)

    # 11. Convert text to lowercase
    text = text.lower()

    # 12. Remove stopwords (requires NLTK library)
    try:
        stop_words = set(stopwords.words('english'))
        text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    except Exception as e:
        print(f"NLTK stopwords error: {e}. Please ensure stopwords are downloaded.")

    # Final removal of extra whitespaces after all operations
    text = re.sub(r'\s+', ' ', text).strip()

    # Final step: Re-insert the specific price ($29.99)
    text = text.replace(target_price_placeholder.lower(), '$29.99') # placeholder will be lowercased by step 12
    # Ensure any remaining placeholders are removed if somehow missed
    text = text.replace(target_price_placeholder, '')

    return text

# Run the cleaning process with the updated function
cleaned_text_with_spellcheck = clean_text(text_data)

# Print the original and newly cleaned text for comparison
print("Original Text:\n", text_data)
print("\n" + "="*50 + "\n")
print("Cleaned Text (with spell check and improved punctuation/price handling):\n", cleaned_text_with_spellcheck)

Original Text:
 
Thís ís à próblemátic téxt fíle!! It contains **extra spaces** ,,,,, special characters!!!💥🔥🚀
Some words are misspelled, and encoding issues liké thís cäusé problëms.
Prices are inconsistent: $29.99, 29.99 USD, 29,99$.
Emails & phone numbers may be embedded: contact@domain.com, (123)-456-7890.
Repeated punctuations!!!!! should be removed, along with **random symbols** like @@,##.
stopwords like "the", "is", and "a" appear often.
HTML tags might be present: <div>This is inside a div</div>
And sometimes, contractions won't expand: "can't", "won't", "shouldn't".
Random numeric values: 123456, 98765, 2024



Cleaned Text (with spell check and improved punctuation/price handling):
 problematic text file contains extra spaces special characters words misspelled encoding issues like problems prices inconsistent $29.99 emails phone numbers may embedded repeated punctuations removed along random symbols like stopwords like appear often tags might present inside sometimes contra

In [ ]:
from google.colab import files

file_name = "cleaned_text.txt"
try:
    files.download(file_name)
except FileNotFoundError:
    print(f"File '{file_name}' not found. Please ensure the cleaning process was run and the file was saved.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Save the cleaned text to a file
file_name = "cleaned_text.txt"
with open(file_name, "w", encoding="utf-8") as f:
    f.write(cleaned_text_with_spellcheck)

print(f"Cleaned text saved to {file_name}")
print("Please download this file again using the download button provided previously.")

Cleaned text saved to cleaned_text.txt
Please download this file again using the download button provided previously.


In [ ]:
from google.colab import files

file_name = "cleaned_text.txt"
try:
    files.download(file_name)
except FileNotFoundError:
    print(f"File '{file_name}' not found. Please ensure the cleaning process was run and the file was saved.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>